# Operaciones básicas con DataFrames

## Descripción de las variables

El dataset, obtenido de <a target = "_blank" href="https://www.transtats.bts.gov/Fields.asp?Table_ID=236">este link</a> está compuesto por las siguientes variables referidas siempre al año 2018:

1. **Month** 1-4
2. **DayofMonth** 1-31
3. **DayOfWeek** 1 (Monday) - 7 (Sunday)
4. **FlightDate** fecha del vuelo
5. **Origin** código IATA del aeropuerto de origen
6. **OriginCity** ciudad donde está el aeropuerto de origen
7. **Dest** código IATA del aeropuerto de destino
8. **DestCity** ciudad donde está el aeropuerto de destino  
9. **DepTime** hora real de salida (local, hhmm)
10. **DepDelay** retraso a la salida, en minutos
11. **ArrTime** hora real de llegada (local, hhmm)
12. **ArrDelay** retraso a la llegada, en minutos: se considera que un vuelo ha llegado "on time" si aterrizó menos de 15 minutos más tarde de la hora prevista en el Computerized Reservations Systems (CRS).
13. **Cancelled** si el vuelo fue cancelado (1 = sí, 0 = no)
14. **CancellationCode** razón de cancelación (A = aparato, B = tiempo atmosférico, C = NAS, D = seguridad)
15. **Diverted** si el vuelo ha sido desviado (1 = sí, 0 = no)
16. **ActualElapsedTime** tiempo real invertido en el vuelo
17. **AirTime** en minutos
18. **Distance** en millas
19. **CarrierDelay** en minutos: El retraso del transportista está bajo el control del transportista aéreo. Ejemplos de sucesos que pueden determinar el retraso del transportista son: limpieza de la aeronave, daño de la aeronave, espera de la llegada de los pasajeros o la tripulación de conexión, equipaje, impacto de un pájaro, carga de equipaje, servicio de comidas, computadora, equipo del transportista, problemas legales de la tripulación (descanso del piloto o acompañante) , daños por mercancías peligrosas, inspección de ingeniería, abastecimiento de combustible, pasajeros discapacitados, tripulación retrasada, servicio de inodoros, mantenimiento, ventas excesivas, servicio de agua potable, denegación de viaje a pasajeros en mal estado, proceso de embarque muy lento, equipaje de mano no válido, retrasos de peso y equilibrio.
20. **WeatherDelay** en minutos: causado por condiciones atmosféricas extremas o peligrosas, previstas o que se han manifestado antes del despegue, durante el viaje, o a la llegada.
21. **NASDelay** en minutos: retraso causado por el National Airspace System (NAS) por motivos como condiciones meteorológicas (perjudiciales pero no extremas), operaciones del aeropuerto, mucho tráfico aéreo, problemas con los controladores aéreos, etc.
22. **SecurityDelay** en minutos: causado por la evacuación de una terminal, re-embarque de un avión debido a brechas en la seguridad, fallos en dispositivos del control de seguridad, colas demasiado largas en el control de seguridad, etc.
23. **LateAircraftDelay** en minutos: debido al propio retraso del avión al llegar, problemas para conseguir aterrizar en un aeropuerto a una hora más tardía de la que estaba prevista.

Leemos el fichero CSV utilizando el delimitador por defecto de Spark (","). La primera línea contiene encabezados (nombres de columnas) por lo que no es parte de los datos y debemos indicarlo con la opción header.

In [1]:
# Esto no hace nada: la lectura es lazy así que no se lee en realidad hasta que ejecutemos una acción sobre flightsDF
# Solamente se comprueba que exista el fichero en esa ruta, y se leen los nombres de columnas
flightsDF = spark.read.option("header", "true")\
                 .csv("gs://data/flights-jan-apr-2018.csv")

In [2]:
# Veamos el esquema (nombre y tipo de dato de cada columna). Esto son solamente metadatos, por lo que no es ninguna acción.
flightsDF.printSchema()

root
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- FlightDate: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCity: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DestCity: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- CarrierDelay: string (nullable = true)
 |-- WeatherDelay: string (nullable = true)
 |-- NASDelay: string (nullable = true)
 |-- SecurityDelay: string (nullable = true)
 |-- LateAircraftDelay: string (nullable = true)



Todas las columnas son cadenas de caracteres porque no hemos indicado el tipo de dato para cada columna ni tampoco le hemos pedido a Spark que intente inferirlo a partir de los datos. No queremos que todas sean string porque hay algunas numéricas que deberían ser tratadas como tales. Vamos a intentar inferir el esquema. Esto supone una lectura un poco más lenta y también es más lento que una tercera opción que consiste en indicar explícitamente el esquema para los datos en el momento de la lectura, que es la opción recomendada si sabemos de antemano qué tipo va a tener cada columna. Si lo hiciésemos de esa manera, en caso de que no se pueda leer con ese esquema obtendríamos un error.

In [3]:
from pyspark.sql import functions as F

flightsDF = spark.read\
                 .option("header", "true")\
                 .option("inferSchema", "true")\
                 .csv("gs://ucmbucket/data/flights-jan-apr-2018.csv") # pon aquí la ruta en tu bucket

# Ensuciamos a propósito la variable ArrDelay para que pase a ser un string como suele pasar con frecuencia
flightsDF = flightsDF.withColumn("ArrDelay",\
                                 F.when(F.rand(123) < 0.1, "NA").otherwise(F.col("ArrDelay")))

flightsDF.printSchema()

root
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- FlightDate: timestamp (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCity: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DestCity: string (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- ArrTime: integer (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- Cancelled: double (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: double (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- CarrierDelay: double (nullable = true)
 |-- WeatherDelay: double (nullable = true)
 |-- NASDelay: double (nullable = true)
 |-- SecurityDelay: double (nullable = true)
 |-- LateAircraftDelay: double (nullable = true)



Ahora tiene mejor pinta, aunque todavía hay algunas columnas cuyo tipo de dato sigue siendo string cuando la intuición nos dice que deberían ser enteros.

## Operaciones básicas con Data Frames

### Consultamos las columnas que tiene el DF

In [4]:
flightsDF.columns

['Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Origin',
 'OriginCity',
 'Dest',
 'DestCity',
 'DepTime',
 'DepDelay',
 'ArrTime',
 'ArrDelay',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'ActualElapsedTime',
 'AirTime',
 'Distance',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay']

### Contamos el número de filas

Es una de las primeras cosas que nos preguntamos sobre un dataset: número de filas y columnas (cuántos ejemplos y cuántas variables tenemos para describirlos). Puesto que vamos a llevar a cabo varias transformaciones al DataFrame `flightsDF` a partir de este punto, vamos a usar `cache`() para que Spark lo mantenga en memoria en lugar de liberar la memoria ocupada tras cada acción.

In [5]:
# Extraemos los nombres de columna. Esto son solo metadatos de DataFrame, y están en el driver. No es necesaria ninguna
# operación sobre el cluster para recuperar la variable interna columns de cualquier DataFrame
print("Los datos tienen {0} columnas".format(len(flightsDF.columns)))

flightsDF.cache()        # Esta línea no hace cálculos, pero Spark anota que debe mantener este DF en memoria tras la primera vez que sea materializado
rows = flightsDF.count() # Esto es una acción que obligará a que flightsDF sea materializado. Para ello, habrá que llevar a cabo
                         # las transformaciones que lo generan en la celda anterior: read y withColumn, que están pendientes

print("Los datos tienen {0} filas".format(rows))

Los datos tienen 23 columnas
Los datos tienen 2503113 filas


In [6]:
flightsDF.count()

2503113

### Seleccionar columnas por nombre

In [7]:
resultDF = flightsDF.select("Month", "DayofMonth", "ArrTime")
resultDF.show(10) # los nombres son sensibles a mayúsculas

+-----+----------+-------+
|Month|DayofMonth|ArrTime|
+-----+----------+-------+
|    1|        14|   null|
|    1|         3|   1506|
|    1|         6|   1543|
|    1|         7|   1455|
|    1|         8|   1509|
|    1|         9|   1504|
|    1|        10|   1455|
|    1|        11|   1452|
|    1|        12|   1748|
|    1|        13|   1514|
+-----+----------+-------+
only showing top 10 rows



### Filtramos (retenemos) filas en base a los valores de una o varias columnas

La mayoría de las transformaciones están definidas en el paquete `pyspark.sql.functions`, por lo que es frecuente importar el paquete completo con un alias, como `F`, en lugar de importar cada función individual. A partir de ese momento usamos `F.` antes del nombre de cada función, para decirle a python dónde buscar esa función.

In [8]:
flightsJanuary20 = None
flightsJanuary20 = flightsDF\
                      .where("DayofMonth = 20 and Month = 1")\
                      .withColumn("DistKm", 1.6 * flightsDF["Distance"])\
                      .withColumn("DobleDistancia", 2 * F.col("DistKm"))
#                      .select("Month", "ArrTime", "DistKm") # encadenamos dos transformaciones: esto no desencadena ninguna operación

In [9]:
flightsJanuary20.printSchema()

root
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- FlightDate: timestamp (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCity: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DestCity: string (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- ArrTime: integer (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- Cancelled: double (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: double (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- CarrierDelay: double (nullable = true)
 |-- WeatherDelay: double (nullable = true)
 |-- NASDelay: double (nullable = true)
 |-- SecurityDelay: double (nullable = true)
 |-- LateAircraftDelay: double (nullable = true)
 |-- DistKm: double (nullable = 

In [10]:
# La función col se utiliza para decir que nos estamos refiriendo a la columna cuyo nombre se pasa como argumento
flightsJanuary20 = flightsDF\
                      .where((F.col("DayofMonth") == 20) & (F.col("Month") == 1))\
                      .select("Month", "ArrTime")\
                      .cache() # encadenamos dos transformaciones: esto no desencadena ninguna operación

# Cúantos vuelos hay el 20 de enero de 2018
# La operación count() es una acción, así que obliga a materializar flightsJanuary20. Para ello es necesario ejecutar
# las transformaciones where() y select() que pusimos en esta celda, aplicadas a flightsDF. De hecho, si no hubiésemos
# cacheado flightsDF en las celdas anteriores, también habría que materializarlo otra vez, y para eso se leería de 
# nuevo el CSV desde HDFS
rowsJanuary20 = flightsJanuary20.count()

print("Hubo {0} vuelos el 20 de enero de 2018".format(rowsJanuary20))

# Esto es otra acción aplicada sobre el DataFrame flightsJanuary. Como flightsJanuary NO ha sido cacheado,
# entonces las operaciones "where" y "select" se necesitan ejecutar DE NUEVO para poder hacer el show()
flightsJanuary20.show(3)

Hubo 16176 vuelos el 20 de enero de 2018
+-----+-------+
|Month|ArrTime|
+-----+-------+
|    1|   1533|
|    1|   1734|
|    1|   2025|
+-----+-------+
only showing top 3 rows



**No olvidemos cachear el DataFrame cuando tengamos previsto hacer varias operaciones sobre él, o de lo contrario estaremos repitiendo muchas veces los cálculos previos que llevaron a ese DataFrame!!**

In [11]:
# También podemos indicar el filtrado como un string con un trozo de código SQL
# Recordemos que where y filter son exactamente equivalentes
flightsJanuary31 = flightsDF.filter("DayofMonth = 31 and Month = 1") # transformación filter: Spark no ejecuta nada

flightsJanuary31.count() # acción count: obliga a materializar flightsJanuary31 para lo cual se tiene que ejecutar filter

20519

<div class="alert alert-block alert-info">
<b>RECUERDA:</b> Spark hace todas estas operaciones de manera distribuida en el cluster, por tanto cada nodo del cluster está filtrando filas de entre aquellas que están presentes en ese nodo (más precisamente, en ese executor). Cada executor envía al driver el recuento de cuántas filas ha filtrado, y los resultados son agregados en el driver para mostrar el recuento total.
</div>

<div class="alert alert-block alert-success">
<b>TU TURNO</b>: muestra por pantalla el retraso en la llegada, el aeropuerto de origen y de destino de aquellos vuelos que tuvieron lugar en Domingo y con un retraso a la llegada mayor de 15 minutos. Muestra el esquema de dicho DataFrame resultante.

</div>

In [12]:
flightsDomingoDF = flightsDF.select("ArrDelay", "Origin", "Dest")\
                            .filter("DayOfWeek = 7 and ArrDelay > 15")
flightsDomingoDF.show()

+--------+------+----+
|ArrDelay|Origin|Dest|
+--------+------+----+
|    21.0|   BOS| BUF|
|    16.0|   BOS| JFK|
|    22.0|   LGA| MSN|
|   136.0|   BOS| JFK|
|   119.0|   DTW| JFK|
|   687.0|   DTW| MBS|
|    90.0|   CVG| DTW|
|    61.0|   GNV| ATL|
|    48.0|   ATL| SHV|
|    45.0|   SHV| ATL|
|   158.0|   MSP| EWR|
|    17.0|   LAS| FLL|
|   238.0|   MCO| BOS|
|    61.0|   BOS| FLL|
|    34.0|   SLC| JFK|
|    80.0|   JFK| PWM|
|    23.0|   MCO| EWR|
|    19.0|   OAK| JFK|
|    63.0|   SJU| BDL|
|    50.0|   LAX| JFK|
+--------+------+----+
only showing top 20 rows



In [13]:
# sintaxis SQL pura: where("DayofWeek = 7 and ArrDelay > 15")
# otra posibilidad: where( (flightsDF["DayofWeek"] == 7) & (flightsDF["ArrDelay"] > 15) )
domingoretraso = flightsDF.where((F.col("DayofWeek") == 7) & (F.col("ArrDelay") > 15))\
                          .select('Origin','Dest','ArrDelay')

domingoretraso.show()

+------+----+--------+
|Origin|Dest|ArrDelay|
+------+----+--------+
|   BOS| BUF|    21.0|
|   BOS| JFK|    16.0|
|   LGA| MSN|    22.0|
|   BOS| JFK|   136.0|
|   DTW| JFK|   119.0|
|   DTW| MBS|   687.0|
|   CVG| DTW|    90.0|
|   GNV| ATL|    61.0|
|   ATL| SHV|    48.0|
|   SHV| ATL|    45.0|
|   MSP| EWR|   158.0|
|   LAS| FLL|    17.0|
|   MCO| BOS|   238.0|
|   BOS| FLL|    61.0|
|   SLC| JFK|    34.0|
|   JFK| PWM|    80.0|
|   MCO| EWR|    23.0|
|   OAK| JFK|    19.0|
|   SJU| BDL|    63.0|
|   LAX| JFK|    50.0|
+------+----+--------+
only showing top 20 rows



### Seleccionar filas únicas

Para hacerse mejor idea de cómo es una variable categórica, es lógico querer cuántos valores distintos existen en nuestro dataset. Si consideramos todas las columnas, sería raro tener dos filas exactamente iguales en todos los valores, pero si seleccionamos solamente una o unas pocas columnas, podemos ver cuántas combinaciones distintas de valores de esas columnas se dan en el dataset.


In [14]:
distinctFlights = flightsDF.distinct()  # distinct es una transformación que devuelve el DF sin las filas repetidas
distinctFlightsCount = distinctFlights.count() # count es una acción y provoca que se ejecute la transformación distinct

print("Hay {0} filas distintas".format(distinctFlightsCount))

Hay 2483784 filas distintas


Si seleccionamos solo las columnas `Origin` y `Dest` y nos quedamos con las filas distintas (quitamos repetidos), entonces obtenemos un DataFrame con los posibles aeropuertos de origen y destino, es decir, aquellos trayectos que existen en un vuelo (pueden aparecer varias veces porque seguramente existirán muchos vuelos a lo largo de 2018 entre un origen y un destino)

<div class="alert alert-block alert-success">
    <b>TU TURNO</b>: ¿<b>cuántas</b> combinaciones de Origin y Dest existen?
</div>

**Respuesta en la siguiente celda:**

In [15]:
countOriginDests = flightsDF.select("Origin", "Dest").distinct().count()

print("Hay {0} combinaciones de un aeropuerto de origen y uno de destino".format(countOriginDests))

Hay 5795 combinaciones de un aeropuerto de origen y uno de destino


Aunque tengamos 2.5 millones de filas, solo hay 5795 combinaciones distintas de un aeropuerto de origen y otro de destino.

<div class="alert alert-block alert-success">
    <b>TU TURNO</b>: ¿<b>cuántos</b> aeropuertos de origen existen?
</div>

In [16]:
distinctOrigins = flightsDF.select("Origin")\
                           .distinct()\
                           .count()
        
flightsDF.select(F.countDistinct("Origin").alias("origDistintos")).show()

print("Hay {0} aeropuertos desde los que puede partir un vuelo".format(distinctOrigins))

+-------------+
|origDistintos|
+-------------+
|          356|
+-------------+

Hay 356 aeropuertos desde los que puede partir un vuelo


<div class="alert alert-block alert-success">
<b>TU TURNO</b>: vamos a hacer esto para cada columna, en un bucle, para hacernos a la idea de cuántos valores hay. Esto solo tiene sentido en realidad para columnas categóricas y no para numéricas donde casi todos los valores serán distintos.
</div>

In [17]:
def dfConteos(sparkDF):
    colsRecuento = [F.countDistinct(c).alias(c) for c in sparkDF.columns]
    return sparkDF.select(colsRecuento)

conteosDF = dfConteos(flightsDF)
conteosDF.show()

+-----+----------+---------+----------+------+----------+----+--------+-------+--------+-------+--------+---------+----------------+--------+-----------------+-------+--------+------------+------------+--------+-------------+-----------------+
|Month|DayofMonth|DayOfWeek|FlightDate|Origin|OriginCity|Dest|DestCity|DepTime|DepDelay|ArrTime|ArrDelay|Cancelled|CancellationCode|Diverted|ActualElapsedTime|AirTime|Distance|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+-----+----------+---------+----------+------+----------+----+--------+-------+--------+-------+--------+---------+----------------+--------+-----------------+-------+--------+------------+------------+--------+-------------+-----------------+
|    4|        31|        7|       120|   356|       350| 356|     350|   1440|    1378|   1440|    1376|        2|               5|       2|              705|    669|    1466|        1094|         748|     609|          128|              739|
+-----+----------+------

In [17]:
for columnName in flightsDF.columns:

    distinctValues = flightsDF.select(columnName).distinct().count()
    
    # No olvidéis indentar este comando para indicar que está dentro del cuerpo del bucle
    print("Existen {0} valores distintos en la columna {1}".format(distinctValues, columnName))


Existen 4 valores distintos en la columna Month
Existen 31 valores distintos en la columna DayofMonth
Existen 7 valores distintos en la columna DayOfWeek
Existen 120 valores distintos en la columna FlightDate
Existen 356 valores distintos en la columna Origin
Existen 350 valores distintos en la columna OriginCity
Existen 356 valores distintos en la columna Dest
Existen 350 valores distintos en la columna DestCity
Existen 1441 valores distintos en la columna DepTime
Existen 1379 valores distintos en la columna DepDelay
Existen 1441 valores distintos en la columna ArrTime
Existen 1377 valores distintos en la columna ArrDelay
Existen 2 valores distintos en la columna Cancelled
Existen 6 valores distintos en la columna CancellationCode
Existen 2 valores distintos en la columna Diverted
Existen 706 valores distintos en la columna ActualElapsedTime
Existen 670 valores distintos en la columna AirTime
Existen 1466 valores distintos en la columna Distance
Existen 1095 valores distintos en la co

### Crear una nueva columna o reemplazar una existente por el resultado de operar con columnas existentes

Tenemos las distancias en millas. Vamos a convertirlas a km multiplicando las millas por 1.61. De nuevo, cada nodo del cluster hará esta operación localmente con los datos que se encuentran en ese nodo. El DataFrame resultante estará repartido en los mismos nodos. **No hay movimiento de datos ya que no se necesita ninguna información que no esté presente en ese nodo** para efectuar la operación.

In [18]:
# withColumn is a transformation returing a new DataFrame with one extra column appended on the right
flightsWithKm = flightsDF.withColumn("DistanceKm", F.col("Distance") * 1.61)

flightsWithKm.printSchema()

flightsWithKm.show(3)

root
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- FlightDate: timestamp (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCity: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DestCity: string (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- ArrTime: integer (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- Cancelled: double (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: double (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- CarrierDelay: double (nullable = true)
 |-- WeatherDelay: double (nullable = true)
 |-- NASDelay: double (nullable = true)
 |-- SecurityDelay: double (nullable = true)
 |-- LateAircraftDelay: double (nullable = true)
 |-- DistanceKm: double (nullabl

El día de la semana es una variable entera. Utilizar un entero o una variable categórica es una decisión que depende de qué tipo de modelo vayamos a ajustar. ¿Tiene sentido considerar días de la semana "más grandes" o "más pequeños", es decir, algo que se incrementa conforme "se incrementa" el día de la semana de 1 a 7? Aquí vamos a **reemplazar** la columna `DayOfWeek` que ya existía, por una versión categórica como strings. Utilizamos `withColumn` pasándole como primer argumento el nombre de una columna que ya existe, lo cual indica que queremos reemplazarla, en el mismo lugar que ocupaba.

In [19]:
flightsCategoricalDay = flightsDF.withColumn("DayOfWeek", F.when(F.col("DayOfWeek") == 1, "Monday")\
                                                           .when(F.col("DayOfWeek") == 2, "Tuesday")\
                                                           .when(F.col("DayOfWeek") == 3, "Wednesday")\
                                                           .when(F.col("DayOfWeek") == 4, "Thursday")\
                                                           .when(F.col("DayOfWeek") == 5, "Friday")\
                                                           .when(F.col("DayOfWeek") == 6, "Saturday")\
                                                           .otherwise("Sunday"))

flightsCategoricalDay.printSchema() # the column is still in the same position but has now string type

flightsCategoricalDay.select("DayOfWeek", "DepTime", "ArrTime").show()

root
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: string (nullable = false)
 |-- FlightDate: timestamp (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCity: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DestCity: string (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- ArrTime: integer (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- Cancelled: double (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: double (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- CarrierDelay: double (nullable = true)
 |-- WeatherDelay: double (nullable = true)
 |-- NASDelay: double (nullable = true)
 |-- SecurityDelay: double (nullable = true)
 |-- LateAircraftDelay: double (nullable = true)

+---------+-------+-------+
|Da

<div class="alert alert-block alert-info">
    <b>CONSEJO:</b> El proceso de crear nuevas variables o <i>features</i> a partir de otras existentes, incluso incorporar variables de fuentes de datos externas (datos públicos) y de limpiar o reemplazar variables tras normalizarla se denomina genéricamente <i>feature engineering</i> (ingeniería de variables). A veces tiene mucho que ver con conocimientos específicos del dominio, aunque también con trucos estadísticos para normalizar siguiend métodos bien conocidos.
</div>

La función `when` es muy común para re-categorizar una variable o para crear nuevas variables categóricas a partir de condiciones complejas acerca de lo que les ocurre a los valores de otras columnas en esa misma fila.

<div class="alert alert-block alert-success">
<b>TU TURNO</b>: crea una variable categórica de tipo string con dos categorías indicando si el día de la semana es laborable o es fin de semana. Los valores deben ser "laborable" o "finde". Utiliza `withColumn` y `when`. Será laborable cuando DayOfWeek esté entre 1 y 5, y fin de semana cuando sea 6 o 7 (en resumen: "en otro caso" es fin de semana). 
</div>

In [20]:
flightsFindeLaborable = flightsDF.withColumn("Laborable", F.when(F.col("DayOfWeek")<= 5, "Laborable")\
                                                           .otherwise("Finde"))

flightsFindeLaborable.printSchema()
flightsFindeLaborable.select("DayOfWeek", "Laborable").show()

root
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- FlightDate: timestamp (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCity: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DestCity: string (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- ArrTime: integer (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- Cancelled: double (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: double (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- CarrierDelay: double (nullable = true)
 |-- WeatherDelay: double (nullable = true)
 |-- NASDelay: double (nullable = true)
 |-- SecurityDelay: double (nullable = true)
 |-- LateAircraftDelay: double (nullable = true)
 |-- Laborable: string (nullable

In [21]:
flightsCategoricalDay = flightsCategoricalDay.withColumn("TypeOfDay", 
                            F.when(~(F.col("DayOfWeek").isin({'Saturday','Sunday'})), "laborable")\
                             .otherwise("finde"))
flightsCategoricalDay.show(3) 

+-----+----------+---------+-------------------+------+------------+----+------------+-------+--------+-------+--------+---------+----------------+--------+-----------------+-------+--------+------------+------------+--------+-------------+-----------------+-----------+
|Month|DayofMonth|DayOfWeek|         FlightDate|Origin|  OriginCity|Dest|    DestCity|DepTime|DepDelay|ArrTime|ArrDelay|Cancelled|CancellationCode|Diverted|ActualElapsedTime|AirTime|Distance|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|  TypeOfDay|
+-----+----------+---------+-------------------+------+------------+----+------------+-------+--------+-------+--------+---------+----------------+--------+-----------------+-------+--------+------------+------------+--------+-------------+-----------------+-----------+
|    1|        14|   Sunday|2018-01-14 00:00:00|   SYR|Syracuse, NY| DTW| Detroit, MI|   null|    null|   null|    null|      1.0|               B|     0.0|             null|   null|   37

In [ ]:
findeDF = flightsDF.withColumn("finde", F.when(F.col("DayOfWeek") <= 5, "laborable")\
                                         .otherwise("finde"))

### Crear y seleccionar columnas al vuelo

`withColumn` no es la única manera de crear columnas. Podemos usar `select` no solo para seleccionar columnas existentes sino también para crear nuevas columnas al vuelo y a la vez seleccionarlas. Vamos a seleccionar Origin y Dest, que ya existen, a la vez que seleccionamos una tercera columna que estamos creando al vuelo, con la transformación de la distancia a km. Le ponemos como nombre "DistanceKm" en ese momento que la estamos creando, mediante la función `alias`. Si no usamos `alias`, Spark le pondrá a la nueva columna un nombre por defecto que en este caso sería "1.6 * Distance". Se recomienda usar alias para dar nombre a las nuevas columnas.

In [20]:
flightsDF.selectExpr("Origin", "Dest", "1.6*Distance AS DistanceKm").show()

+------+----+----------+
|Origin|Dest|DistanceKm|
+------+----+----------+
|   SYR| DTW|     598.4|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
+------+----+----------+
only showing top 20 rows



In [22]:
flightsAirportsAndKm = flightsDF.select("Origin",\
                                        "Dest",\
                                        (1.6 * F.col("Distance")).alias("DistanceKm"))
flightsAirportsAndKm.show()

+------+----+----------+
|Origin|Dest|DistanceKm|
+------+----+----------+
|   SYR| DTW|     598.4|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
+------+----+----------+
only showing top 20 rows



In [21]:
flightsAirportsAndKm = flightsDF.select(F.col("Origin"),\
                                        F.col("Dest"),\
                                        (1.6 * F.col("Distance")).alias("DistanceKm"))

flightsAirportsAndKm.show(5)

+------+----+----------+
|Origin|Dest|DistanceKm|
+------+----+----------+
|   SYR| DTW|     598.4|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
|   SYR| LGA|     316.8|
+------+----+----------+
only showing top 5 rows



<div class="alert alert-block alert-success">
<b>TU TURNO</b>: crear un DataFrame con tres columnas seleccionando "Origin", "OriginCity" y una nueva columna de tipo string creada concatenando Origin y OriginCity con un guión "-". Utilizar `withColumn` y dentro la función `concat_ws` (concatenar con separador) con sintaxis <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.concat_ws">F.concat_ws("-", columna1, columna2)</a> del paquete pyspark.sql.functions.
</div>

In [ ]:
flightsConcat = flightsDF.withColumn("Concat",F.concat_ws("-","Origin","OriginCity"))\
                                     .select("Origin","OriginCity", "Concat")
flightsConcat.show(5)

In [29]:
concatenadoDF = flightsDF.withColumn("concatenado", F.concat_ws("-", F.col("Origin"), F.col("OriginCity")))\
                         .select(F.col("Origin"), F.col("OriginCity"), F.col("concatenado"))
concatenadoDF.show()

+------+------------+----------------+
|Origin|  OriginCity|     concatenado|
+------+------------+----------------+
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
+------+------------+----------------+
only showing top 20 rows



In [25]:
concatenadoDF = flightsDF.select(F.col("Origin"), F.col("OriginCity"),
                                 F.concat_ws("-", F.col("Origin"), F.col("OriginCity")).alias("concatenado"))
concatenadoDF.show()

+------+------------+----------------+
|Origin|  OriginCity|     concatenado|
+------+------------+----------------+
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
|   SYR|Syracuse, NY|SYR-Syracuse, NY|
+------+------------+----------------+
only showing top 20 rows



### Convertir el tipo de dato de una columna

Con frecuencia, Spark no infiere correctamente el tipo de cada columna. 
* A veces ocurre que una columna numérica no se reconozca adecuadamente porque los datos tienen "NA" (un string) que quería significar data faltante en el dataset original. "NA" no es un string especial para Spark, por lo que simplemente reconoce que la columna tiene tanto enteros como strings, y el tipo más general que infiere es string para esa columna. 
* Pero no es el caso aquí porque no se da esto, y por eso hemos ensuciado al principio los datos a propósito. Vamos a limpiarlos y a convertir la columna `ArrDelay` a DoubleType como debe ser.

In [26]:
naCount = flightsDF.where("ArrDelay = 'NA'").count()

# Esto es sintaxis SQL, pero también podríamos haberla llamado como .where(F.col("ArrDelay") == "NA"). Ambas son equivalentes.

print("Hay ", naCount, "filas con NA en ArrDelay")

('Hay ', 249841, 'filas con NA en ArrDelay')


In [27]:
flightsDF.where(F.col("ArrDelay") != "NA").count()

2196457

In [28]:
flightsDF.is_cached

True

In [30]:
#from pyspark.sql.types import DoubleType
from pyspark.sql import types as T

flightsDF = flightsDF.where((F.col("ArrDelay") != "NA"))\
                     .withColumn("ArrDelay", F.col("ArrDelay").cast(T.DoubleType()))

In [31]:
flightsDF.is_cached

False

### Ordenación respecto a una columna

Es una transformación que nos devuelve otro DataFrame ordenado según la(s) columna(s) indicada(s), sea de forma ascendente o descendente. El DataFrame original no se modifica (al igual que ocurre en cualquier transformación). Se puede ordenar en base a una columna numérica (lo más frecuente) o también categórica (los strings se ordenan alfabéticamente). Para ordenar sí es necesario enviar información de unos nodos a otros puesto que no sabemos si existen o no datos mayores o menos que el valor que tenemos en el nodo (o más precisamente, en el worker).

In [32]:
# Ordenamos los vuelos según ArrDelay
sortedDF = flightsDF.orderBy("ArrDelay")  # equivalente: flightsDF.orderBy(F.col("ArrDelay"))

# Orden ascendentemente por aeropuerto de origen ("Origin") y deshago los empates por arr_delay descendentemente
sortedDF = flightsDF.orderBy(F.col("Origin"), F.col("ArrDelay").desc())  # equivalente: flightsDF.orderBy(F.col("ArrDelay"))

sortedDescDF = flightsDF.orderBy("ArrDelay", ascending = False)
sortedDescDF.select("ArrDelay", "Origin", "Dest").show(10)

+--------+------+----+
|ArrDelay|Origin|Dest|
+--------+------+----+
|  2475.0|   HNL| PPG|
|  2454.0|   PPG| HNL|
|  2023.0|   EGE| JFK|
|  1778.0|   ORF| DFW|
|  1757.0|   SMF| DFW|
|  1717.0|   HNL| JFK|
|  1685.0|   ORF| JFK|
|  1650.0|   ABQ| DFW|
|  1648.0|   SLC| DFW|
|  1576.0|   IAH| MIA|
+--------+------+----+
only showing top 10 rows



In [33]:
primeraFila = sortedDescDF.first()

In [37]:
primeraFila.ArrTime

1445

In [35]:
primeraFila.asDict()

{'Month': 2,
 'DayofMonth': 9,
 'DayOfWeek': 5,
 'FlightDate': datetime.datetime(2018, 2, 9, 0, 0),
 'Origin': 'HNL',
 'OriginCity': 'Honolulu, HI',
 'Dest': 'PPG',
 'DestCity': 'Pago Pago, TT',
 'DepTime': 1002,
 'DepDelay': 2482.0,
 'ArrTime': 1445,
 'ArrDelay': 2475.0,
 'Cancelled': 0.0,
 'CancellationCode': None,
 'Diverted': 0.0,
 'ActualElapsedTime': 343.0,
 'AirTime': 305.0,
 'Distance': 2599.0,
 'CarrierDelay': 0.0,
 'WeatherDelay': 2475.0,
 'NASDelay': 0.0,
 'SecurityDelay': 0.0,
 'LateAircraftDelay': 0.0}

In [39]:
listaRows = sortedDescDF.take(20)  # devuelve una lista de Python, de objetos Row
distancias = [r.Distance for r in listaRows]  # usamos sintaxis de listas por comprensión
distancias

[2599.0,
 2599.0,
 1746.0,
 1212.0,
 1431.0,
 4983.0,
 290.0,
 569.0,
 989.0,
 964.0,
 861.0,
 1810.0,
 282.0,
 2504.0,
 1235.0,
 1363.0,
 228.0,
 1172.0,
 861.0,
 761.0]

In [40]:
listaLAX = flightsDF.where("Origin = 'LAX'").collect() # ¡¡¡¡ CUIDADO !!!!
len(listaLAX)

71647

In [41]:
listaLAX[0:4]

[Row(Month=1, DayofMonth=1, DayOfWeek=1, FlightDate=datetime.datetime(2018, 1, 1, 0, 0), Origin='LAX', OriginCity='Los Angeles, CA', Dest='JFK', DestCity='New York, NY', DepTime=600, DepDelay=0.0, ArrTime=1414, ArrDelay=-2.0, Cancelled=0.0, CancellationCode=None, Diverted=0.0, ActualElapsedTime=314.0, AirTime=285.0, Distance=2475.0, CarrierDelay=None, WeatherDelay=None, NASDelay=None, SecurityDelay=None, LateAircraftDelay=None),
 Row(Month=1, DayofMonth=1, DayOfWeek=1, FlightDate=datetime.datetime(2018, 1, 1, 0, 0), Origin='LAX', OriginCity='Los Angeles, CA', Dest='FLL', DestCity='Fort Lauderdale, FL', DepTime=2120, DepDelay=10.0, ArrTime=452, ArrDelay=-5.0, Cancelled=0.0, CancellationCode=None, Diverted=0.0, ActualElapsedTime=272.0, AirTime=250.0, Distance=2343.0, CarrierDelay=None, WeatherDelay=None, NASDelay=None, SecurityDelay=None, LateAircraftDelay=None),
 Row(Month=1, DayofMonth=1, DayOfWeek=1, FlightDate=datetime.datetime(2018, 1, 1, 0, 0), Origin='LAX', OriginCity='Los Angeles

### Vamos a comprobar que al llevar al driver las 10 primeras filas, el DF estaba ordenado así que esas filas están ordenadas

In [34]:
lista = ordenadoDF.take(10) # esto devuelve una lista de python

In [35]:
lista

[Row(ArrDelay=674.0, Origin=u'ABE', Dest=u'DTW'),
 Row(ArrDelay=581.0, Origin=u'ABE', Dest=u'DTW'),
 Row(ArrDelay=547.0, Origin=u'ABE', Dest=u'DTW'),
 Row(ArrDelay=505.0, Origin=u'ABE', Dest=u'DTW'),
 Row(ArrDelay=486.0, Origin=u'ABE', Dest=u'PGD'),
 Row(ArrDelay=459.0, Origin=u'ABE', Dest=u'ATL'),
 Row(ArrDelay=445.0, Origin=u'ABE', Dest=u'ORD'),
 Row(ArrDelay=394.0, Origin=u'ABE', Dest=u'PGD'),
 Row(ArrDelay=387.0, Origin=u'ABE', Dest=u'ORD'),
 Row(ArrDelay=385.0, Origin=u'ABE', Dest=u'PHL')]

### Funciones de agregación sobre el DataFrame completo

Spark tiene implementaciones distribuidas y paralelas de funciones de agregación frecuentes como la media, min, max y desviación típica entre otras. Todas estas funciones reciben como argumento el objeto columna sobre el que la función debe aplicarse. Lo más habitual es aplicarlas para agregar por grupos, pero esto lo veremos en el segundo notebook.

Vamos a seleccionar vuelos con `ArrDelay` mayor de 15 minutos, y con ellos vamos a crear columnas con el min, max, media y desviación típica de la columna `ArrDelay`. Crearemos y seleccionaremos dichas columnas al vuelo, como vimos en la sección anterior.

In [ ]:
flightsDF.printSchema()

In [ ]:
flightsDF.count()

In [46]:
# First we select those flights with at least 16 minutes of delay and then compute the aggregations
agregacionesDF = flightsDF.where(F.col("ArrDelay") > 15)\
                          .selectariance("ArrDelay").alias("varArrDelay")
                            )

agregacionesDF.show()

+-----------------+-----------+-----------+----------------+-----+-----------------+
|     MeanArrDelay|MinArrDelay|MaxArrDelay|  StddevArrDelay|nOrig|      varArrDelay|
+-----------------+-----------+-----------+----------------+-----+-----------------+
|65.14031947595987|       16.0|     2475.0|82.9741567315577|  356|6884.710685313103|
+-----------------+-----------+-----------+----------------+-----+-----------------+



Existe una función de Spark que hace casi todo esto por nosotros, llamada `summary`. Lo hace para cada columna numérica que encuentre en el dataset. Es también una transformación que devuelve un nuevo DataFrame con las métricas de resumen, sin modificar el DataFrame original.

In [47]:
summariesDF = flightsDF.summary().cache()  # habría sido buena idea hacer flightsDF.summary().cache()
summariesDF.show()

+-------+------------------+------------------+-----------------+-------+------------+-------+------------+------------------+------------------+------------------+-----------------+---------+----------------+--------+------------------+------------------+-----------------+------------------+------------------+------------------+-------------------+------------------+
|summary|             Month|        DayofMonth|        DayOfWeek| Origin|  OriginCity|   Dest|    DestCity|           DepTime|          DepDelay|           ArrTime|         ArrDelay|Cancelled|CancellationCode|Diverted| ActualElapsedTime|           AirTime|         Distance|      CarrierDelay|      WeatherDelay|          NASDelay|      SecurityDelay| LateAircraftDelay|
+-------+------------------+------------------+-----------------+-------+------------+-------+------------+------------------+------------------+------------------+-----------------+---------+----------------+--------+------------------+------------------+--

In [64]:
summariesDF.printSchema()

root
 |-- summary: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCity: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DestCity: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- CarrierDelay: string (nullable = true)
 |-- WeatherDelay: string (nullable = true)
 |-- NASDelay: string (nullable = true)
 |-- SecurityDelay: string (nullable = true)
 |-- LateAircraftDelay: string (nullable = true)



In [65]:
print(summariesDF.count()) # vamos a contar el número de filas del DF resumen

8
['summary', 'Month', 'DayofMonth', 'DayOfWeek', 'Origin', 'OriginCity', 'Dest', 'DestCity', 'DepTime', 'DepDelay', 'ArrTime', 'ArrDelay', 'Cancelled', 'CancellationCode', 'Diverted', 'ActualElapsedTime', 'AirTime', 'Distance', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']


### Conversión a Pandas

En general es complicado leer los resultados que muestra Spark. En ocasiones interesa convertirlos a un dataframe de Pandas, aunque esto implica traer todas las filas al driver, lo cual **debe hacerse con mucho cuidado y solo en casos en los que estemos seguros de que el DataFrame de Spark es pequeño y no desbordará la memoria del driver**.

<div class="alert alert-block alert-info">
<p><b>CONSEJO</b>: el concepto de dataframe como tabla con filas y columnas existe en muchos lenguajes de programación (pandas de Python, dataframes de R, DataFrames de Spark). Sin embargo, Spark maneja DataFrames que están físicamente distribuidos en las memorias RAM de las máquinas del cluster, lo cual no tiene nada que ver con lo que hace la biblioteca Pandas o R que se ejecutan en una sola máquina. Convertir un DataFrame de Spark en un dataframe de Pandas implica llevar todas las filas al driver, lo cual podría resultar en una excepción Out-of-Memory si el contenido del DataFrame es más grande que la memoria RAM de la máquina en la que se está ejecutando el programa driver. En este caso y en la mayoría de casos, se suele utilizar para mostrar resúmenes o agregados ya calculados previamente, y que sabemos que ocupan poco, con lo que no existe riesgo.</p>

<p>Esta operación también es muy frecuente cuando queremos representar gráficamente el contenido de un DataFrame de Spark. No existen funciones gráficas en Spark, por lo que tenemos que convertirlo en un dataframe de Pandas y utilizar las funciones gráficas de Python habituales (matplotlib, Seaborn o incluso la propia biblioteca Pandas) para mostrar lo que necesitemos.
</p>
</div>

In [69]:
summariesDF.is_cached

False

In [38]:
flightsPd = summariesDF.toPandas()

In [67]:
flightsPd

,summary,Month,DayofMonth,DayOfWeek,Origin,OriginCity,Dest,DestCity,DepTime,DepDelay,...,CancellationCode,Diverted,ActualElapsedTime,AirTime,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,count,2195756,2195756,2195756,2195756,2195756,2195756,2195756,2195756,2193859,...,0,2195756,2195754,2193103,2195756,391595,391595,391595,391595,391595
1,mean,2.5428845463703618,15.647962706238763,3.9119838451995577,None,None,None,None,1334.6682564000737,8.77650113339098,...,None,0.0,133.29546980217273,108.46217938692347,766.7551672408046,20.08061900688211,3.3975357193018296,14.406716122524546,0.08250360704299084,25.316081666006973
2,stddev,1.1248089885230748,8.70562480253341,2.0043659206090956,None,None,None,None,498.6958329189281,43.464185589574306,...,None,0.0,71.9919447145365,70.20722828449857,583.9065781219789,60.56429305377853,30.855181839720377,32.37040004800845,2.526014171447696,49.95008971538814
3,min,1,1,1,ABE,"Aberdeen, SD",ABE,"Aberdeen, SD",1,-611.0,...,None,0.0,11.0,5.0,16.0,0.0,0.0,0.0,0.0,0.0
4,25%,2,8,2,None,None,None,None,920,-6.0,...,None,0.0,82.0,58.0,337.0,0.0,0.0,0.0,0.0,0.0
5,50%,3,16,4,None,None,None,None,1329,-3.0,...,None,0.0,115.0,90.0,604.0,1.0,0.0,2.0,0.0,2.0
6,75%,4,23,6,None,None,None,None,1741,6.0,...,None,0.0,164.0,138.0,1009.0,17.0,0.0,19.0,0.0,31.0
7,max,4,31,7,YUM,"Yuma, AZ",YUM,"Yuma, AZ",2400,2482.0,...,None,0.0,757.0,696.0,4983.0,2007.0,2475.0,1392.0,593.0,2454.0


In [39]:
otroDF = spark.createDataFrame(flightsPd)